In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from google.colab import drive

# Mount Google Drive to access data
drive.mount('/content/drive')

# Load data from Google Drive (Update path if needed)
df_train = pd.read_csv('/content/drive/MyDrive/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/test.csv')

# Display basic info
print(df_train.info())
print(df_train.describe())

# Feature Engineering
df_train['Cabin_Deck'] = df_train['Cabin'].apply(lambda x: str(x).split('/')[0])
df_train['Cabin_Side'] = df_train['Cabin'].apply(lambda x: str(x).split('/')[-1])
df_test['Cabin_Deck'] = df_test['Cabin'].apply(lambda x: str(x).split('/')[0])
df_test['Cabin_Side'] = df_test['Cabin'].apply(lambda x: str(x).split('/')[-1])

# Convert categorical features to numerical
label_encoders = {}
categorical_cols = ['HomePlanet', 'Destination', 'Cabin_Deck', 'Cabin_Side']
for col in categorical_cols:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])
    df_test[col] = le.transform(df_test[col])
    label_encoders[col] = le

# Define features and target variable
X = df_train.drop(['PassengerId', 'Name', 'Cabin', 'Transported'], axis=1)
y = df_train['Transported'].astype(int)
X_test = df_test.drop(['PassengerId', 'Name', 'Cabin'], axis=1)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Model selection and training
rf = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=42)
rf.fit(X_train, y_train)

# Predictions
y_pred = rf.predict(X_val)
y_prob = rf.predict_proba(X_val)[:, 1]

# Evaluation
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)
print(f'Accuracy: {accuracy:.4f}, ROC AUC: {roc_auc:.4f}')
print(classification_report(y_val, y_pred))

# Generate submission
submission = pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Transported': rf.predict(X_test)})
submission.to_csv('/content/drive/MyDrive/submission.csv', index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
None
        